# Semantic Search on Specific Data Corpus
Query files within specifi corpus.

In [0]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Set up Azure OpenAI

In [0]:
!pip install -r /Workspace/Repos/jeanpierre.vanniekerk@truenorthgroup.co.za/document-analysis-using-gpt-3.ide/notebooks/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 14.1 MB/s eta 0:00:00
  Using cached openai-0.27.8-py3-none-any.whl (73 kB)
  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/15.2 MB 5.0 MB/

*** WARNING: max output size exceeded, skipping output. ***

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 8.3 MB/s eta 0:00:00
  Using cached msal-1.23.0-py2.py3-none-any.whl (90 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.6/520.6 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.5 MB/s eta 0:00:00
  Using cached tzlocal-5.0.1-py3-none-any.whl (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.7/239.7 kB 21.3 MB/s eta 0:00:00
  U

In [0]:
import os
import openai
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv()
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

False

## Deploy a Language Model

In [0]:
# list models deployed with embeddings capability
deployment_id = None
result = openai.Deployment.list()

for deployment in result.data:
    if deployment["status"] != "succeeded":
        continue
    
    model = openai.Model.retrieve(deployment["model"])
    if model["capabilities"]["embeddings"] != True:
        continue
    
    deployment_id = deployment["id"]
    break

# if not model deployed, deploy one
if not deployment_id:
    print('No deployment with status: succeeded found.')
    model = "text-similarity-davinci-001"

    # Now let's create the deployment
    print(f'Creating a new deployment with model: {model}')
    result = openai.Deployment.create(model=model, scale_settings={"scale_type":"standard"})
    deployment_id = result["id"]
    print(f'Successfully created {model} with deployment_id {deployment_id}')
else:
    print(f'Found a succeeded deployment that supports embeddings with id: {deployment_id}.')

Found a succeeded deployment that supports embeddings with id: text-embedding-ada-002.


## Create Embeddings

see [01-get-embeddings.ipynb](./01-get-embeddings.ipynb) on how to get embeddings.

In this example, we will load embeddings from a file.

## Load Data

In [0]:
import pandas as pd
df_orig = spark.sql('select * from openai.document_analysis_embeddings').limit(100).toPandas()

In [0]:
import numpy as np

DEVELOPMENT = False

if DEVELOPMENT:
    # Sub-sample for development
    df = df_orig.sample(n=20, replace=False, random_state=9).copy()
else:
    df = df_orig.copy()

# drop rows with NaN
df.dropna(inplace=True)

# convert string to array
df["embedding"] = df['embedding'].apply(np.array)#.apply(eval)
df

,category,filename,title,content,embedding,n_tokens
0,business,001.txt,Ad sales boost Time Warner profit,""" Quarterly profits at US media giant TimeWarn...","[-0.01796060986816883, -0.01651429571211338, 0...",563
1,business,002.txt,Dollar gains on Greenspan speech,""" The dollar has hit its highest level against...","[-0.021637994796037674, -0.012706875801086426,...",457
2,business,003.txt,Yukos unit buyer faces loan claim,""" The owners of embattled Russian oil giant Yu...","[-0.01746140420436859, -0.03939249739050865, 0...",340
3,business,004.txt,High fuel prices hit BA's profits,""" British Airways has blamed high fuel prices ...","[-0.019902830943465233, -0.017122669145464897,...",558
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,"[-0.008347506634891033, -0.008401579223573208,...",385
...,...,...,...,...,...,...
95,business,096.txt,Burren awarded Egyptian contracts,""" British energy firm Burren Energy has been a...","[-0.005239638965576887, -0.023387936875224113,...",180
96,business,097.txt,'Strong dollar' call halts slide,""" The US dollar's slide against the euro and y...","[-0.04523226246237755, -0.03272397071123123, 0...",558
97,business,098.txt,IMF 'cuts' German growth estimate,""" The International Monetary Fund is to cut it...","[-0.01721673458814621, -0.019378477707505226, ...",285
98,business,099.txt,"GM, Ford cut output as sales fall",""" US car firms General Motors (GM) and Ford ha...","[-0.026180488988757133, -0.013377368450164795,...",551


## Find documents with similar embeddings to the embeddings of the question

In [0]:
import numpy as np

def get_embedding(text, deployment_id=deployment_id):
    """ 
    Get embeddings for an input text. 
    """
    result = openai.Embedding.create(
      deployment_id=deployment_id,
      input=text
    )
    result = np.array(result["data"][0]["embedding"])
    return result

def vector_similarity(x, y):
    """
    Returns the similarity between two vectors.    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    similarity = np.dot(x, y)
    return similarity 

def order_document_sections_by_query_similarity(query, contexts):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)

    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

## Retrieve relevant news

In [0]:
def retrieve_relevant_documents(query, contexts = df['embedding']):
    # find text most similar to the query
    answers = order_document_sections_by_query_similarity(query=query, contexts=contexts)[0:3]

    # print top 3
    for answer in answers:
        print(f'similarity score:   {answer[0]}')
        print(df['content'].loc[answer[1]], '\n')

    return

In [0]:
query = 'News about stock market.'
retrieve_relevant_documents(query=query)

similarity score:   0.8173208606631346
" Irish shares have risen to a record high, with investors persuaded to buy into the market by low inflation and strong growth forecasts.  The ISEQ index of leading shares closed up 23 points to 6661.89 on Thursday, fuelled by strong growth in banking and financial stocks. A fall in the rate of inflation to 2.3% in January gave a fresh boost to shares which have advanced 4% this month. The economy is set for strong growth in 2005 while interest rates remain low.  Several of Ireland's biggest companies saw their market value hit recent highs on Thursday. Allied Irish Banks, Ireland's biggest company by capitalisation, touched a five year peak while Bank of Ireland shares rose to their highest level since August 2002.  Telecoms firm Eircom, which recently revealed that it would re-enter the Irish mobile phone market, hit a yearly high. Analysts said that economic conditions were benign and Irish shares were still trading at a discount to other Europ

In [0]:
query = 'What is happening in the rugby world?'
retrieve_relevant_documents(query=query)

similarity score:   0.7602209318297704
" UK sportswear firm Umbro has posted a 222% rise in annual profit after sales of replica England football kits were boosted by the Euro 2004 tournament.  Pre-tax profit for 2004 was £15.4m ($29.4m). Umbro, which recently lost sponsorship deals with Chelsea and Celtic, said on Thursday it had signed a new four-year agreement with Scottish club Rangers. It hopes 2005 sales will benefit from the launch of a new England replica shirt ahead of the 2006 World Cup. In January, Umbro announced its sponsorship agreement with Chelsea, which gave Umbro the lucrative right to make replica shirts, would end in 2006, five years earlier than expected. The firm, which is to receive a payment from Chelsea of £24.5m, said it is ""appraising a number of additional investment opportunities as a result of this compensation"" . Chief executive Peter McGuigan said the firm plans to grow sales both in the UK and internationally.  The firm, reporting its first annual res

In [0]:
query = 'What happened in Brazil?'
retrieve_relevant_documents(query=query)

similarity score:   0.7832974227000485
 Argentina and Venezuela have extended a food-for-oil deal, which helped the former to overcome a severe energy crisis last year.  Argentine President Nestor Kirchner and Venezuelan President Hugo Chavez signed the deal in Buenos Aires on Tuesday. Last April, Argentina signed a $240m agreement to import Venezuelan fuel in exchange for agricultural goods and this deal has now been extended. Venezuela will now import cattle, medicines and medical equipment. Last year, Argentina's severe energy crisis forced President Kirchner to suspend gas exports to Chile. Argentina fears that rising demand could spark another crisis and wants to prevent it by signing this deal. The two countries also formalised a co-operation deal between Venezuelan energy firm PDVSA and Argentina's Enarsa. Under this deal, the Argentine market will be opened to Venezuelan investment. President Chavez added that Brazil's Petrobras could join soon the co-operation deal. President 